# Tutorial of Theano

Source: http://deeplearning.net/software/theano/tutorial/index.html

## Preparation
Import Theano packages and other necessary packages

In [1]:
from theano import *
import theano.tensor as T
import numpy as np

## Data type declaration

In Theano, all symbols must be typed.
In particular, `T.dscalar` is the type we assign to
“0-dimensional arrays (`scalar`) of doubles (`d`)”.
It is a Theano Type.

`dscalar` is not a class.
Therefore, neither `x` nor `y` are actually instances of `dscalar`.
They are instances of `TensorVariable`
assigned the Theano Type `dscalar` in their type field,
as shown here:

In [2]:
x = T.dscalar('x')
y = T.dscalar('y')
print(type(x))
print(x.type)
print(T.dscalar)
print(x.type is T.dscalar)

<class 'theano.tensor.var.TensorVariable'>
TensorType(float64, scalar)
TensorType(float64, scalar)
True


Theano uses expression to describe the relation between variables.
Now a new variable `z` is defined to be the summation of `x` and `y`.
Use Theano `pp` to pretty-print `z`.

In [3]:
z = x + y
pp(z)

'(x + y)'

Theano uses the keyword `function` to name such relationship.
The first argument to function is a list of variables served as function input.
The second argument is a single Variable or a list of Variables as function output.
`f` may then be used like a normal Python function.



In [4]:
f = function([x, y], z)
f(2,3)

array(5.)

## Extension to matrix data types

`dmatrix` is the Type for matrices of doubles.
Now we define our variables to be matrices.

In [5]:
x = T.dmatrix('x')
y = T.dmatrix('y')
print(type(x))
print(x.type)
print(T.dmatrix)
print(x.type is T.dmatrix)

<class 'theano.tensor.var.TensorVariable'>
TensorType(float64, matrix)
TensorType(float64, matrix)
True


In [6]:
z = x + y
f = function([x, y], z)

# addition of two vectors
f([[2,3]], [[4,5]])

array([[6., 8.]])

In [7]:
# addition of two matrices
f([[2,3],[4,5]], [[6,7],[8,9]])

array([[ 8., 10.],
       [12., 14.]])

It is also available to use `numpy` array data type as function input.

In [8]:
# addition of two numpy.array matrices
f(np.array([[2,3],[4,5]]), np.array([[6,7],[8,9]]))

array([[ 8., 10.],
       [12., 14.]])

Now we could also try other types such as `vector`.
The following function tries to compute $a^2 + b^2 + 2ab$.

In [9]:
a = T.vector()
b = T.vector()
out = a**2 + b**2 + 2*a*b
f = function([a, b], out)
f([0,1,2], [10,11,12])

array([100., 144., 196.])

## Logistic function

The following function computes the sigmoid.
$$s(x) = \frac{1}{1+e^{-x}}$$
Note that Theano has math library including `exp()`.

In [10]:
x = T.dmatrix()
s = 1/(1+T.exp(-x))
sigmoid = function([x], s)
sigmoid([[0, 1], [-1, -2]])

array([[0.5       , 0.73105858],
       [0.26894142, 0.11920292]])

## Function with multiple outputs

The following function computes summation, difference, and inner product of two vectors.

In [11]:
a = T.vector()
b = T.vector()
summ = a+b
diff = abs(a-b)
inpd = T.dot(a, b.T)
f = function([a, b], [summ, diff, inpd])
f([0,1,2], [10,11,12])

[array([10., 12., 14.]), array([10., 10., 10.]), array(35.)]

## Setting default values of variables
We can use `In` class to specify properties of your function’s parameters with greater detail.
Here we give a default value of 1 for y by creating a `In` instance with its `value` field set to 1.

In [12]:
x, y = T.dscalars('x', 'y')
z = x + y
# set a default value 1 for y
f = function([x, In(y, value=1)], z)
# show addition with default y
print(f(33))
# show results with other y value
print(f(33, 5))

34.0
38.0


These parameters can be set positionally or by `name`, 
so that we can specified in a function call.

In [15]:
x, y = T.dscalars('x', 'y')
z = x + y
# set a default value 1 for y
f = function([In(x, value=0, name="x_value"), In(y, value=1, name="y_value")], z)
f(y_value=3, x_value=2)

array(5.)

## Using Shared Variables
It is also possible to make a function with an internal state. 
For example, let’s say we want to make an accumulator: 
at the beginning, the state is initialized to zero.
Then, on each function call, the state is incremented by the function’s argument.

First let’s define the `accumulator` function.
It adds its argument to the internal state, and returns the old state value.

Here we have a few new concepts.
The `shared` function constructs so-called shared variables.
These are hybrid symbolic and non-symbolic variables whose value may be shared between multiple functions.
Shared variables can be used in symbolic expressions just like the objects returned by `dmatrices(...)`
but they also have an internal value that defines
the value taken by this symbolic variable in all the functions that use it. 
It is called a shared variable because its value is shared between many functions.
The value can be accessed and modified by the `.get_value()` and `.set_value()` methods. 

The other new thing in this code is the `updates` parameter of function.
`updates` must be supplied with a list of pairs of the form (shared-variable, new expression).
It can also be a dictionary whose keys are shared-variables and values are the new expressions.
Either way, it means “whenever this function runs, it will replace the `.value` of each shared variable
with the result of the corresponding expression”.

Our accumulator replaces the `state`‘s value with the sum of the state and the increment amount.

In [19]:
state = shared(0)
inc = T.iscalar('inc')
accumulator = function([inc], state, updates=[(state, state+inc)])

print("Time 0: state = ", state.get_value())
print("Accumulator with 1 as input, current value = ", accumulator(1))
print("Time 1: state = ", state.get_value())
print("Accumulator with 300 as input, current value = ", accumulator(300))
print("Time 2: state = ", state.get_value())

Time 0: state =  0
Accumulator with 1 as input, current value =  0
Time 1: state =  1
Accumulator with 300 as input, current value =  1
Time 2: state =  301
